In [4]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

## GET INDEKS LINK

In [269]:
def getIndeksLink(links, page, offset, cat_link, category, date=datetime.strftime(datetime.today(), '%Y/%m/%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY-mm-dd
    """
    print("page ", page)
    url = "http://"+cat_link+".metrotvnews.com/index/"+date+"/"+ str(offset)
    print(url)
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    contentDiv = soup.find('div', class_="style_06").findAll('h2')
    contentDiv
    for post in contentDiv:
        link = [post.find('a',href=True)['href']]
        links.append(link)
            
    el_page = soup.find('div', class_="grid")
    if el_page:
        a_page = el_page.findAll('div', class_='bu fr')[-1].find('a')
        max_page = int(a_page['data-ci-pagination-page'].replace('\n', '').strip(' '))
        
        if page < max_page:
            links = getIndeksLink(links, page+1, offset+30, cat_link, category, date)    
        
    return links

## GET DETAIL BERITA

In [280]:
def getDetailBerita(url):
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        #print(url)
        response = requests.get(url)
        html = response.text

        #Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")

        #extract subcategory from breadcrumb
        bc = soup.find('div', class_="breadcrumbs")
        if not bc:
            continue
        cat = bc.findAll('a')[-2].text
        sub = bc.findAll('a')[-1].text

        #articles
        article_id = int(soup.find('meta', attrs={"property":"og:image"})['content'].replace('//','').split('/')[6])
        articles['id'] = article_id

        #category
        #category
        articles['category'] = cat
        articles['subcategory'] = sub
        
        articles['url'] = url

        article = soup.find('div', class_="tru")

        #extract date
        pubdate_author = soup.find("div",class_="reg").text
        pubdate_author_split = pubdate_author.split(' \xa0\xa0 • \xa0\xa0 ')
        pubdate = pubdate_author_split[1]
        pubdate = pubdate.strip(' ')
        pubdate = pubdate.replace(' WIB','')
        pubdate = datetime.strftime(datetime.strptime(pubdate, "%A, %d %b %Y %H:%M"), "%Y-%m-%d %H:%M:%S")
        articles['pubdate'] = pubdate

        #extract author
        author = pubdate_author_split[0]
        articles['author'] = author

        #extract title
        articles['title'] = soup.find('meta', attrs={"property":"og:title"})['content']
        if ("foto" in sub.lower()) or  "video" in sub.lower(): 
            continue
            
        #source
        articles['source'] = 'metrotvnews'

        #extract comments count
        articles['comment'] = 0

        #extract tags
        tags = soup.find('div', class_="line").findAll('a', class_="tag")
        articles['tags'] = ','.join([x.text for x in tags])

        #extract images
        articles['images'] = soup.find('img', class_="pic")['src']

        #extract detail
        detail = soup.find('div', class_="tru")

        #hapus link sisip
        for link in detail.findAll('div', class_="related"):
            link.decompose()

        #hapus video sisip
        for tag in detail.findAll('iframe', class_="embedv"):
            tag.decompose()

        #hapus all setelah clear fix
        #for det in detail.find('div', class_="wfull fl rl"):
        #    det.decompose()

        #hapus all script
        for script in detail.findAll('script'):
            script.decompose()

        #extract content
        detail = BeautifulSoup(detail.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
        articles['content'] = content.strip(' ')
        #print('memasukkan berita id ', articles['id'])
        all_articles.append(articles)
    return all_articles

In [277]:
links = getIndeksLink([], 1, 0, 'news', 'news', '2018/07/01')

page  1
http://news.metrotvnews.com/index/2018/07/01/0
page  2
http://news.metrotvnews.com/index/2018/07/01/30


In [278]:
links

[['http://news.metrotvnews.com/politik/0k8Zed0N-akbar-tanjung-putusan-mk-harus-dipatuhi'],
 ['http://news.metrotvnews.com/politik/ob3VLyJN-larangan-eks-koruptor-nyaleg-dinilai-membatasi-pilihan-rakyat'],
 ['http://news.metrotvnews.com/peristiwa/zNAwnRzk-ini-dia-7-penerima-hkti-innovation-award-2018'],
 ['http://news.metrotvnews.com/politik/GKdWelek-akbar-tanjung-minta-kalla-mempersiapkan-diri-jika-maju-pilpres'],
 ['http://news.metrotvnews.com/metro/Wb7JmQdN-rambu-perluasan-ganjil-genap-dipasang-jelang-uji-coba'],
 ['http://news.metrotvnews.com/peristiwa/1bVGar2k-tambah-daya-tampung-dki-bangun-42-smk-baru'],
 ['http://news.metrotvnews.com/peristiwa/0Kv7DYGN-mendikbud-dorong-dki-jadi-percontohan-revitalisasi-smk'],
 ['http://news.metrotvnews.com/metro/lKYE5DAK-uji-coba-perluasan-ganjil-genap-diberlakukan-besok'],
 ['http://news.metrotvnews.com/politik/GbmjnPxk-golkar-tak-ngotot-airlangga-mendampingi-jokowi'],
 ['http://news.metrotvnews.com/politik/nbw7wPmb-pemuda-muhammadiyah-koruptor-s

In [284]:
details = getDetailBerita(links[0:3])
details

[{'author': ' Intan Yunelia',
  'category': 'News',
  'comment': 0,
  'content': 'Jakarta: Wakil Ketua Dewan Kehormatan Partai Golkar Akbar Tanjung meminta semua pihak menghormati putusan Mahkamah Konstitusi (MK) soal durasi masa jabatan presiden dan wakil presiden. Putusan itu menutup peluang Jusuf Kalla (JK) maju kembali sebagai calon wakil presiden.\xa0  “Kita semua mengetahui bahwa keputusan MK itu final dan mengikat, jadi kalau tidak ada representative, itu ya mestinya harus kita ikuti keputusan MK itu,” kata Akbar di Nusantara V Kompleks MPR RI, Jalan Gatot Subroto, Jakarta Pusat, Minggu, 1 Juli 2018.     Mantan ketua umum Partai Golkar ini menyebutk, MK bisa melakukan uji materi apabila memang ada bukti-bukti yang memperkuat. Menurut dia, aturan ini sebetulnya sudah jelas, namun ada pihak-pihak yang ingin JK kembali maju sebagai cawapres.\xa0  “Kalau berkaitan kan dua kali berturut-turut tapi karena mungkin ada perbedaan penafsiran antara orang orang di lingkungan Pak JK," ucap 

In [285]:
df = pd.DataFrame(details)

In [286]:
df

,author,category,comment,content,id,images,pubdate,source,subcategory,tags,title,url
0,Intan Yunelia,News,0,Jakarta: Wakil Ketua Dewan Kehormatan Partai G...,895369,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-01 23:33:00,metrotvnews,Politik,pilpres 2019,Akbar Tanjung: Putusan MK Harus Dipatuhi,http://news.metrotvnews.com/politik/0k8Zed0N-a...
1,Intan Yunelia,News,0,Jakarta: Wakil Ketua Dewan Pembina Partai Golk...,895357,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-01 22:11:00,metrotvnews,Politik,pemilu serentak 2019,Larangan Eks Koruptor Nyaleg Dinilai Membatasi...,http://news.metrotvnews.com/politik/ob3VLyJN-l...
2,Gervin Nathaniel Purba,News,0,Jakarta: Himpunan Kerukunan Tani Indonesia (HK...,895346,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-01 21:10:00,metrotvnews,Peristiwa,HKTI,Ini Dia 7 Penerima HKTI Innovation Award 2018,http://news.metrotvnews.com/peristiwa/zNAwnRzk...
3,Intan Yunelia,News,0,Jakarta: Wakil Ketua Dewan Kehormatan Partai G...,895334,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-01 20:10:00,metrotvnews,Politik,pilpres 2019,Akbar Tanjung Minta Kalla Mempersiapkan Diri j...,http://news.metrotvnews.com/politik/GKdWelek-a...
4,Sunnaholomi Halakrispen,News,0,Jakarta: Kasubdit Gakkum Ditlantas Polda Metro...,895333,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-01 20:05:00,metrotvnews,Metro,sistem ganjil genap,Rambu Perluasan Ganjil-Genap Dipasang Jelang U...,http://news.metrotvnews.com/metro/Wb7JmQdN-ram...
5,Intan Yunelia,News,0,Jakarta: Pemerintah Provinsi DKI Jakarta akan...,895326,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-01 18:38:00,metrotvnews,Peristiwa,Revitalisasi SMK,"Tambah Daya Tampung, DKI Bangun 42 SMK Baru",http://news.metrotvnews.com/peristiwa/1bVGar2k...
6,Intan Yunelia,News,0,Jakarta: Menteri Pendidikan dan Kebudayaan (M...,895320,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-01 18:23:00,metrotvnews,Peristiwa,Revitalisasi SMK,Mendikbud Dorong DKI Jadi Percontohan Revitali...,http://news.metrotvnews.com/peristiwa/0Kv7DYGN...
7,Nur Azizah,News,0,Jakarta: Uji coba perluasan ruas ganjil genap ...,895317,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-01 18:18:00,metrotvnews,Metro,sistem ganjil genap,Uji Coba Perluasan Ganjil Genap Diberlakukan B...,http://news.metrotvnews.com/metro/lKYE5DAK-uji...
8,Intan Yunelia,News,0,Jakarta: Wakil Ketua Dewan Pembina Partai Golk...,895311,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-01 17:40:00,metrotvnews,Politik,pilpres 2019,Golkar tak Ngotot Airlangga Mendampingi Jokowi,http://news.metrotvnews.com/politik/GbmjnPxk-g...
9,Intan Yunelia,News,0,Jakarta: Ketua Umum Pemuda Muhammadiyah Dahnil...,895309,http://cdn.metrotvnews.com/dynamic/content/201...,2018-07-01 17:33:00,metrotvnews,Politik,pileg,Pemuda Muhammadiyah: Koruptor Seperti Residivis,http://news.metrotvnews.com/politik/nbw7wPmb-p...
